# Aurite Agents Tutorial 2: Giving Agents Tools with MCP Servers

Welcome to the second tutorial in the Aurite Agents series! In the previous tutorial, we learned how to create and configure a basic agent. Now, we'll explore one of the most powerful features of the framework: giving agents **Tools** to interact with the outside world.

In Aurite, tools are provided by **Model Context Protocol (MCP) Servers**. These are external programs that can do anything from searching the web to analyzing data. By connecting an agent to an MCP Server, you give it access to all the tools that server provides (you can also exclude specific tools that you don't want agents to acess when you are adding new mcp server configurations).

### What You'll Learn

In this notebook, you will learn:

1.  **What an MCP Server is** and how to configure one using a `ClientConfig` object.
2.  How to connect to a tool server that runs on the web (using the `http_stream` transport), which is perfect for cloud environments like Google Colab.
3.  How to use the library of **pre-packaged tools** that comes built-in with the `aurite` package.

## Section 1: Setup

### Step 1.1: Install Dependencies

First, let's install the `aurite` package.

In [21]:
%pip install aurite

Note: you may need to restart the kernel to use updated packages.


### Step 1.2: Configure OpenAI API Key

Like in our previous notebooks, we need to configure our OpenAI API key. The cell below will prompt you to enter your key securely.

In [ ]:
import os
from getpass import getpass

# Store the OpenAI API key in the notebook
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# check for the API key to ensure it's set
if not os.environ["OPENAI_API_KEY"]:
    raise ValueError("OpenAI API key is required. Please set the OPENAI_API_KEY environment variable.")

print("✅ OpenAI API key is set.")

✅ OpenAI API key is set.


### Step 1.3: Configure Smithery API Keys

Many of the pre-packaged tools in the Aurite toolbox are hosted on the **Smithery.ai** platform. Smithery makes it easy to deploy and share MCP servers. To use these tools, you'll need a free API key from their platform.

1.  Go to [Smithery.ai](https://smithery.ai/) and create a free account.
2.  Navigate to your profile to find your **API Key** and **Profile ID**.

The cell below will prompt you to enter these keys so the framework can use them.

In [29]:
if "SMITHERY_API_KEY" not in os.environ:
    os.environ["SMITHERY_API_KEY"] = getpass("Enter your Smithery API key: ")
if "SMITHERY_PROFILE_ID" not in os.environ:
    os.environ["SMITHERY_PROFILE_ID"] = getpass("Enter your Smithery Profile ID: ")

# check for the Smithery API key and Profile ID to ensure they're set
if not os.environ["SMITHERY_API_KEY"] or not os.environ["SMITHERY_PROFILE_ID"]:
    raise ValueError("Smithery API key and Profile ID are required. Please set the SMITHERY_API_KEY and SMITHERY_PROFILE_ID environment variables`.")

print("✅ Smithery API key and Profile ID are set.")

✅ Smithery API key and Profile ID are set.


## Step 1.4: Initialize Aurite

Like we did in tutorial 1, we first need to create and initialize the main Aurite application.

In [32]:
from aurite import Aurite

# Create the main Aurite application object
aurite = Aurite()

# Initialize the Aurite application
await aurite.initialize()

INFO     [aurite.config.component_manager] User project config directory not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/config. No project-specific components will be loaded.
WARNING  [aurite.config.project_manager] Project configuration file not found: /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/aurite_config.json
WARNING  [aurite.config.project_manager] Project file not found at /home/wilcoxr/workspace/aurite/aurite-agents/docs/notebooks/aurite_config.json. Creating an empty project named 'aurite_config.json'.
INFO     [aurite.host.host] MCP Host initialization attempt finished. Successfully initialized 0/0 configured clients. 
INFO     [aurite.host_manager] Aurite initialization complete.


## Section 2: Register a New MCP Server

Before we use a tool, it's important to understand how Aurite connects to MCP Servers. This is done using a **`ClientConfig`** object.

### What is a `ClientConfig`?

A `ClientConfig` is a configuration object (which can be defined in Python or JSON) that tells Aurite everything it needs to know to connect to and use an MCP Server. 

Let's look at the configuration for a real tool server from the packaged toolbox: `game_trends_mcp`. This server provides tools for getting data about video games.

```json
{
    "name": "game_trends_mcp",
    "http_endpoint": "https://server.smithery.ai/@halismertkir/game-trends-mcp/mcp?profile={SMITHERY_PROFILE_ID}&api_key={SMITHERY_API_KEY}",
    "capabilities": ["tools"]
}
```

**Key Fields:**
-   `name`: A unique name for the server.
-   `http_endpoint`: The URL of the server. Notice the `{SMITHERY_PROFILE_ID}` and `{SMITHERY_API_KEY}` placeholders—Aurite will automatically replace these with the environment variables you just set!
-   `capabilities`: What the server provides. In this case, `"tools"` (code to execute). Servers can also provide `"prompts"` (text to read), or `"resources'` (data to read).

> For a full breakdown of all configuration options, including other transport types like `stdio` for local scripts, see the **[Client Configurations Documentation](../components/mcp_server.md)**.

## Section 3: Manually Registering a Tool Server

To fully understand the process, we will first walk through all the steps of registering and using a tool server manually.

### Step 3.1: Define the `ClientConfig` in Python

Instead of using a JSON file, we can define the `ClientConfig` directly in our Python code. This makes the concept very clear.

In [ ]:
from aurite.config.config_models import ClientConfig

# Define the configuration for the game trends server
game_trends_config = ClientConfig(
    name="game_trends_mcp",
    http_endpoint=f"https://server.smithery.ai/@halismertkir/game-trends-mcp/mcp?profile={os.environ['SMITHERY_PROFILE_ID']}&api_key={os.environ['SMITHERY_API_KEY']}",
    capabilities=["tools"]
)

# Register the game trends server configuration with Aurite
await aurite.register_client(game_trends_config)

print(f"✅ Successfully registered ClientConfig for: {game_trends_config.name}")

INFO     [aurite.host.host] Attempting to dynamically register client: game_trends_mcp
INFO     [httpx] HTTP Request: POST https://server.smithery.ai/@halismertkir/game-trends-mcp/mcp?profile=chosen-fish-NF9IH0&api_key=7b3772d4-59c3-4995-bfa6-47d7f88f717c "HTTP/1.1 200 OK"
INFO     [mcp.client.streamable_http] Received session ID: eyJpIjoiZTgyMjM2ZWY3NDM5MDgiLCJzIjoiMDE5NzUxNjgtNTg1Ni03MjkyLWE0OTQtYzhmMTZhNjFhODZkIiwidiI6MX0.PbmR7OzQNs_tB_jZygR4ZMZUvdNOjhR9-KStubx6gfY,90c03f3f-a5e8-4a3e-ad1e-960a11f337a1
INFO     [httpx] HTTP Request: POST https://server.smithery.ai/@halismertkir/game-trends-mcp/mcp?profile=chosen-fish-NF9IH0&api_key=7b3772d4-59c3-4995-bfa6-47d7f88f717c "HTTP/1.1 202 Accepted"
INFO     [httpx] HTTP Request: POST https://server.smithery.ai/@halismertkir/game-trends-mcp/mcp?profile=chosen-fish-NF9IH0&api_key=7b3772d4-59c3-4995-bfa6-47d7f88f717c "HTTP/1.1 200 OK"
INFO     [aurite.host.host] Client 'game_trends_mcp' dynamically registered and initialized successfully.


✅ Successfully registered ClientConfig for: game_trends_mcp


INFO     [httpx] HTTP Request: POST https://server.smithery.ai/@halismertkir/game-trends-mcp/mcp?profile=chosen-fish-NF9IH0&api_key=7b3772d4-59c3-4995-bfa6-47d7f88f717c "HTTP/1.1 200 OK"
INFO     [httpx] HTTP Request: GET https://server.smithery.ai/@halismertkir/game-trends-mcp/mcp?profile=chosen-fish-NF9IH0&api_key=7b3772d4-59c3-4995-bfa6-47d7f88f717c "HTTP/1.1 524 <none>"


### Step 3.2: Define an Agent to Use the Tool

Now, let's create an agent and tell it that it's allowed to use the server we just configured. We do this by adding the server's `name` to the agent's `mcp_servers` list.

In [35]:
from aurite.config.config_models import AgentConfig

# Define the agent that will use the game trends tool
analyst_agent_config = AgentConfig(
    name="Game Trend Analyst",
    system_prompt="You are an expert analyst in the video game industry. Use the tools provided to answer questions about game trends.",
    mcp_servers=["game_trends_mcp"] # This links the agent to our ClientConfig
)

await aurite.register_agent(analyst_agent_config)
print(f"✅ Successfully Registered: {analyst_agent_config.name}")

✅ Successfully Registered: Game Trend Analyst


### Step 3.4: Run the Agent

Now for the fun part! Let's ask our agent a question that requires it to use the game trends tool.

In [36]:
from IPython.display import display, HTML
user_query = "What are the top-selling games on Steam right now?"
print(f"Running agent with query: '{user_query}'...")

agent_result = await aurite.run_agent(
    agent_name="Game Trend Analyst",
    user_message=user_query
)

# Render the agent's response and thought process as HTML
display(HTML(agent_result.primary_text))

WARNING  [aurite.execution.facade] Facade: Agent 'Game Trend Analyst' running without a specific LLMConfig ID. Creating temporary, non-cached client using default OpenAI config.
INFO     [aurite.components.llm.providers.openai_client] OpenAIClient initialized for model gpt-4-turbo-preview using direct API calls.
INFO     [aurite.execution.facade] Facade: Running conversation for Aurite Agent 'Game Trend Analyst'...


Running agent with query: 'What are the top-selling games on Steam right now?'...


INFO     [httpx] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO     [aurite.host.host] Executing tool 'get_steam_top_sellers' on client 'game_trends_mcp' for agent 'Game Trend Analyst'
INFO     [httpx] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO     [aurite.execution.facade] Facade: Aurite Agent 'Game Trend Analyst' conversation finished.


## Section 4: The Easy Way - Using the Built-in Toolbox

Manually defining the `ClientConfig` is great for understanding how things work, but Aurite provides a much easier way to use the dozens of tools included in its toolbox: **Just-in-Time (JIT) Registration**.

You don't need to define a `ClientConfig` for any packaged tool. You just need to reference it by name in your agent's `mcp_servers` list, and Aurite will find it and set it up for you automatically.

### Step 4.1: Define a New Agent

Let's create a new agent to be a weather forecaster. We'll tell it to use the `national_weather_service` tool server. Notice that this time, we are **not** creating a `ClientConfig` for it.

In [ ]:
# Define the agent that will use the weather tool
weather_agent_config = AgentConfig(
    name="Weather Forecaster",
    system_prompt="You are a helpful weather assistant. Use the tools provided to get the weather forecast.",
    mcp_servers=["national_weather_service"] # Just reference the packaged tool by name!
)

print(f"Successfully created AgentConfig for: {weather_agent_config.name}")

### Step 4.2: Run the New Agent

Now, we will use our *same* `aurite` instance. We just need to register our new agent, and Aurite will handle the rest.

In [ ]:
async def run_auto_agent():
    # Register just the new agent. Aurite will find and register the tool for us.
    await aurite.register_agent(weather_agent_config)

    user_query = "What's the weather forecast for New York, NY?"
    print(f"Running agent with query: '{user_query}'...")

    agent_result = await aurite.run_agent(
        agent_name="Weather Forecaster",
        user_message=user_query
    )

    display(HTML(agent_result.primary_text))

asyncio.run(run_auto_agent())

### Step 4.3: How It Works

It just works! When you registered the `Weather Forecaster` agent, Aurite's `HostManager` saw that it needed an MCP server named `national_weather_service`. It searched the built-in toolbox, found the corresponding `ClientConfig`, loaded it, and automatically connected to the server using the Smithery API keys you provided earlier. 

This makes using the entire library of packaged tools incredibly simple.

## Section 5: Your Turn to Explore

Now that you know how to use packaged tools, you can experiment with the entire toolbox.

1.  Explore the full list of available tools in the **[Packaged MCP Servers Directory](../toolbox/mcp_server_directory.md)**.
2.  Click on a category to see the available servers and the tools they provide.
3.  Try creating a new agent that uses a different server, like `pubmed_mcp_server` (for medical research) or `appinsightmcp` (for app store data).

**Challenge:** Create a new agent named "Medical Researcher" that uses the `pubmed_mcp_server` and ask it to `"Search for recent articles about AI in medicine."`

## Conclusion

Congratulations! You've learned the fundamentals of giving agents tools in the Aurite framework. 

You now know:
-   How to define and register a `ClientConfig` to connect to any `http_stream` MCP server.
-   How to leverage the powerful Just-in-Time registration to use any of the dozens of tools in the built-in toolbox just by referencing their name.

This opens up a world of possibilities for creating highly capable and specialized agents.